In [ ]:
!pip install transformers==4.28.0 sacremoses sentencepiece sentence-transformers datasets accelerate faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import re
import nltk
import string
import pandas as pd
nltk.download('stopwords')
stopwords_lst = nltk.corpus.stopwords.words('english')
from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
fs = pd.read_csv("/content/drive/MyDrive/Lights-on-heights/data/feature_store.csv")

In [ ]:
fs.head(3)

,Unnamed: 0,drug_information,label,drug_name
0,0,doxycycline miscellaneous antimalarials tetrac...,acne,doxycycline
1,1,spironolactone aldosterone receptor antagonist...,acne,spironolactone
2,2,minocycline tetracyclines dynacin minocin mino...,acne,minocycline


In [ ]:
# GET THE DRUG INFORMATION COLUMN
documents = fs["drug_information"].to_list()

In [ ]:
documents[:3]

['doxycycline miscellaneous antimalarials tetracyclines acticlate adoxa ck adoxa pak adoxa tt alodox avidoxy doryx mondoxyne nl monodox morgidox okebo oracea oraxyl periostat targadox vibramycin calcium vibramycin hyclate vibramycin monohydrate vibra tabs acne amoxicillin prednisone albuterol ciprofloxacin azithromycin cephalexin metronidazole metronidazole topical clindamycin topical clindamycin',
 'spironolactone aldosterone receptor antagonists potassium sparing diuretics aldactone carospir acne amlodipine lisinopril losartan metoprolol furosemide hydrochlorothiazide carvedilol warfarin lasix bumetanide',
 'minocycline tetracyclines dynacin minocin minolira solodyn ximino vectrin myrac acne amoxicillin prednisone doxycycline ciprofloxacin azithromycin cephalexin metronidazole clindamycin topical augmentin dexamethasone']

### CREATE EMBEDDING - USING A PRETRAINED MODEL

In [ ]:
#help(SentenceTransformer)

##### Using the [multi-qa-MiniLM-L6-cos-v1](https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1) model. This model has been trained  on 213 Million Question and  Answer pairs.


In [ ]:
sentence_embedder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') # 'multi-qa-MiniLM-L6-cos-v1' 'distilbert-base-nli-mean-tokens'

In [ ]:
# CREATE THE EMBEDDING LAYER/MATRIX
embeddings = sentence_embedder.encode(documents)

In [ ]:
type(embeddings)

numpy.ndarray

In [ ]:
embeddings.shape

(999, 384)

#### VISUALIZE EMBEDDING

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import plotly.express as px

In [ ]:
visualize_labels = fs["label"].to_list()
visualize_labels = np.asarray(visualize_labels)

In [ ]:
# CREATE A TWO DIMENTIONAL T-SNE PROJECTION OF THE EMBEDDING
tsne = TSNE(2, verbose=1)
tsne_proj = tsne.fit_transform(embeddings)


fig = px.scatter(x=tsne_proj[:, 0], y=tsne_proj[:, 1], color=visualize_labels)
fig.update_layout(
    title="2D t-SNE visualization of embeddings",
    xaxis_title="First t-SNE",
    yaxis_title="Second t-SNE",
)
fig.show()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 999 samples in 0.001s...
[t-SNE] Computed neighbors for 999 samples in 0.132s...
[t-SNE] Computed conditional probabilities for sample 999 / 999
[t-SNE] Mean sigma: 0.349884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.105614
[t-SNE] KL divergence after 1000 iterations: 0.530326


In [ ]:
# CREATE A THREE DIMENTIONAL T-SNE PROJECTION OF THE EMBEDDING
tsne = TSNE(3, verbose=1)
tsne_proj = tsne.fit_transform(embeddings)

fig = px.scatter_3d(x=tsne_proj[:, 0], y=tsne_proj[:, 1], z=tsne_proj[:, 2], color=visualize_labels, opacity=0.8)
fig.update_layout(
    title="3D t-SNE visualization of embeddings",
)
fig.show()

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 999 samples in 0.001s...
[t-SNE] Computed neighbors for 999 samples in 0.121s...
[t-SNE] Computed conditional probabilities for sample 999 / 999
[t-SNE] Mean sigma: 0.349884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.412231
[t-SNE] KL divergence after 1000 iterations: 0.459333


### PERFORM SEMANTIC SEARCH TEST

In [ ]:
import random

In [ ]:
# FUNCTION USED TO CLEAN THE INPUT QUERY
def clean_input_qst(qst):
  string.punctation = string.punctuation
  qst = ' '.join([word for word in qst.split() if word not in (stopwords_lst)])
  qst = re.sub('[%s]' % re.escape(string.punctuation), ' ' , qst)
  qst = qst.lower()
  return qst.strip()

In [ ]:
qst = "what do you have on cold syrup?"

In [ ]:
qst = clean_input_qst(qst)
qst

'cold syrup'

In [ ]:
# ENCODE THE QUERY
qst_embedding = sentence_embedder.encode(qst)

##### The semantic_search function uses cosine similarity to check how similar the question_embedding is to the main embeddings. How this works is, the encoder creates a vector embedding of the question and places it in the vector space, it then finds which of the clusters the query vector is closest to using cosine similarity. The smallest distance signifies how close/similar that query is to that sub vector space. The search returns a list of the courps_id, i.e the id of the text document corresponding to its vector in the embeddings and the score, which shows the confidence level of how close the question vector is to the provided vector embedding .

In [ ]:
resp = util.semantic_search(qst_embedding, embeddings)
resp  = resp[0] 

In [ ]:
 # Get the first k results for the query
k = 6

In [ ]:
resp = resp[:k]

In [ ]:
resp

[{'corpus_id': 970, 'score': 0.525941014289856},
 {'corpus_id': 889, 'score': 0.45283401012420654},
 {'corpus_id': 903, 'score': 0.4288380742073059},
 {'corpus_id': 925, 'score': 0.42351841926574707},
 {'corpus_id': 845, 'score': 0.4150329828262329},
 {'corpus_id': 915, 'score': 0.3833625912666321}]

In [ ]:
# print the actual drug_name. For the schema.
for i in range(len(resp)):
  print(f"DRUG NAME: {fs._get_value(resp[i]['corpus_id'], 'drug_name')}")
  print(f"MEDICAL CONDITION FOR DRUG: {fs._get_value(resp[i]['corpus_id'], 'label')}")
  print("")

DRUG NAME: Codimal DM
MEDICAL CONDITION FOR DRUG: cold

DRUG NAME: Childrens Tylenol Cold Plus Cough
MEDICAL CONDITION FOR DRUG: cold

DRUG NAME: dextromethorphan, phenylephrine, triprolidine
MEDICAL CONDITION FOR DRUG: cold

DRUG NAME: Theraflu Warming Relief Nighttime Severe Cold & Cough
MEDICAL CONDITION FOR DRUG: cold

DRUG NAME: Alka-Seltzer Plus Cold Formula Sparkling Original Effervescent Tablets
MEDICAL CONDITION FOR DRUG: cold

DRUG NAME: Poly Hist Forte
MEDICAL CONDITION FOR DRUG: cold



In [ ]:
def pre_proceess_text(df):
  string.punctation = string.punctuation + "——"
  df = re.sub('\w*\d\w*', '', str(df)) # remove numbers
  df = re.sub(' +', ' ', str(df)) # remove extra whitespaces
  df = re.sub('[%s]' % re.escape(string.punctuation), ' ' , str(df)) # remove punctuations
  df = re.sub('’s', '',str(df))
  df = df.lower() # convert to lower case
  df = df.strip()
  return df

In [ ]:
# SIMULATE REAL QUERIES
def query_shuffler(drug_name):
  query = [f"what do you have on {drug_name}?",f"i need information on {drug_name}.",f"{drug_name}?", f"{drug_name}"]
  query = random.choice(query)
  return query

##### CREATING A MINI TEST SET

In [ ]:
fs[fs["label"]== "aids"].head(15)

,Unnamed: 0,drug_information,label,drug_name
293,293,truvada emtricitabine tenofovir disoproxil fum...,aids,Truvada
294,294,biktarvy bictegravir emtricitabine tenofovir a...,aids,Biktarvy
295,295,triumeq abacavir dolutegravir lamivudine antiv...,aids,Triumeq
296,296,genvoya cobicistat elvitegravir emtricitabine ...,aids,Genvoya
297,297,stribild cobicistat elvitegravir emtricitabine...,aids,Stribild
298,298,atripla efavirenz emtricitabine tenofovir anti...,aids,Atripla
299,299,descovy emtricitabine tenofovir alafenamide an...,aids,Descovy
300,300,norvir ritonavir antiviral boosters protease i...,aids,Norvir
301,301,isentress raltegravir integrase strand transfe...,aids,Isentress
302,302,dovato dolutegravir lamivudine antiviral combi...,aids,Dovato


In [ ]:
drug_names_condtions= ["gemfibrozil","Xyzal","amlodipine","Vicks VapoRub","triamcinolone","Equaline Sleep Aid","Lescol XL","Namenda","Monodox","Depakote Sprinkles","Bayer Aspirin","Truvada","bronchitis","Acticlate","cancer","Dxevo","Namzaric","Depakote ER","Benadryl","Tivicay"]
labels = ["cholesterol","allergies","angina","cold","asthma","allrgies","cholesterol","acne","Namenda","bipolar","angina","aids","bronchitis","acne","cancer","asthma","alzheimer","bipolar","cold","aids"]

In [ ]:
len(drug_names_condtions)

20

In [ ]:
save_test_result = []

In [ ]:
for drug_name in drug_names_condtions:
  drug_name = pre_proceess_text(drug_name)
  question = query_shuffler(drug_name) # simulatr real queries
  print(question)
  qst = clean_input_qst(question)
  qst_embedding = sentence_embedder.encode(qst)
  resp = util.semantic_search(qst_embedding, embeddings)
  resp  = resp[0]
  resp = resp[:1]
  for i in range(len(resp)):
    df_result = fs._get_value(resp[i]['corpus_id'], 'drug_name')
    save_test_result.append(fs._get_value(resp[i]['corpus_id'], 'label'))
    print(df_result)
    print("")

what do you have on gemfibrozil?
gemfibrozil

i need information on xyzal.
Xyzal

amlodipine
amlodipine, atorvastatin

what do you have on vicks vaporub?
Vicks VapoSteam

what do you have on triamcinolone?
Kenalog-40

equaline sleep aid?
Equaline Sleep Aid

lescol xl
Lescol XL

what do you have on namenda?
Namenda XR

monodox?
Monodox

what do you have on depakote sprinkles?
Depakote Sprinkles

i need information on bayer aspirin.
Bayer Aspirin Regimen

what do you have on truvada?
Truvada

i need information on bronchitis.
cefprozil

what do you have on acticlate?
corticotropin

what do you have on cancer?
Leukeran

dxevo
doxorubicin

namzaric
Namzaric

i need information on depakote er.
Depakote ER

i need information on benadryl.
Benadryl Allergy

tivicay?
Tivicay



In [ ]:
assert len(save_test_result) == len(labels)

In [ ]:
count = 0
for i in range(len(save_test_result)):
  if save_test_result[i] == labels[i]:
    count+=1
print(count)

15


In [ ]:
accuracy = count/ len(save_test_result)

In [ ]:
print(f"Accuracy on curated dataset: {accuracy * 100} %")

Accuracy on curated dataset: 75.0 %


### SERIALIZE EMBEDDER


In [ ]:
import pickle

In [ ]:
embedding_local_path = "/content/drive/MyDrive/Lights-on-heights/artefacts/pre-trained_embedder.pkl"

In [ ]:
with open(embedding_local_path, "wb") as f:
  pickle.dump( {"pre_trained_embeddings": embeddings,"pre_trained_embedder":sentence_embedder}, f)